In [1]:
#Importing the Labraries
import pandas as pd

In [2]:
# Import pandas and load diamonds.csv
df = pd.read_csv('/content/drive/MyDrive/Study/Self Study/Swayam/diamond.csv')

In [3]:
df.head(5)

,carat,cut,color,clarity,price
0,0.23,Ideal,E,SI2,326
1,0.21,Premium,E,SI1,326
2,0.23,Good,E,VS1,327
3,0.29,Premium,I,VS2,334
4,0.31,Good,J,SI2,335


In [4]:
#We Have Total 4 features 1 of them is numerical and 3 is categorical feature
#We ahve 53940 Data points
df.info()

(53940, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   price    53940 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 2.1+ MB


In [5]:
df['cut'].value_counts()
#cut feature has 5 types of values and Ideal is most famous

Ideal        21551
Premium      13791
Very Good    12082
Good          4906
Fair          1610
Name: cut, dtype: int64

In [6]:
df['color'].value_counts()
#color feature has 7 types of values and G is most famous

G    11292
E     9797
F     9542
H     8304
D     6775
I     5422
J     2808
Name: color, dtype: int64

In [7]:
df['clarity'].value_counts()
#cut feature has 8 types of values and SI1 is most famous

SI1     13065
VS2     12258
SI2      9194
VS1      8171
VVS2     5066
VVS1     3655
IF       1790
I1        741
Name: clarity, dtype: int64

In [8]:
#Now Let's check for Missing values
df.isnull().sum()

#We don't have any missing values in our data set, but we will be assuming we have and accordingly write code

carat      0
cut        0
color      0
clarity    0
price      0
dtype: int64

In [10]:
# Let's Split the data into independent and target variables 
# Prep the data for a machine learning model that regresses price on the other features in the DataFrame:
y = df['price']
X = df.drop('price', axis=1)

# Now Let's Split the data into Training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [49]:
# Now we will be doing some preprocessing
# Import some preprocessors to deal with those features in the X matrices 
# that contain missing and categorical variables
from sklearn.impute import SimpleImputer as SI
from sklearn.preprocessing import LabelBinarizer as LB
from sklearn.preprocessing import StandardScaler as SS

'''We have techniques LabelEncoder, LabelBinarizer, OneHotEncoder to deal with categorical values
   Label Encoder Simply gives Number to categories
   LabelBinarzier and OneHotEncoder converts them into array of 0's and 1's that represents a particular category
   Both LB and OHE are used for same task but they internally works differently 
   One difference is that LB does not works with Multi column data while OHE works 
    '''
'''StandardScaler is used for scaling the features, we have standardization (values are standarized around the mean) 
   and normalization also known as Min Max Scaling (scaling down values between 0 to 1) 
'''

# Use a DataFrameMapper to organize the preprocessing feature transformations in a single object:
from sklearn_pandas import DataFrameMapper

mapper = DataFrameMapper([
    (['cut'], [SI(strategy='most_frequent'), LB()]),
    (['color'], [SI(strategy='most_frequent'), LB()]),
    (['clarity'], [SI(strategy='most_frequent'), LB()]),
    (['carat'], [SI(strategy='mean'), SS()]),
], df_out=True)

# from sklearn.preprocessing import OneHotEncoder as OHE
# mapper = DataFrameMapper([
#         (['cut', 'color', 'clarity'], [SI(strategy='most_frequent'), OHE()]),
#         (['carat'], [SI(strategy = 'mean'), SS()])], df_out = True)

# DataFrameMapper chains together a list of feature transformations for each 
# column in a single place. The object behaves as though 
# it's a scikit-learn transformer which is a real bonus!

# Fit the mapper and transform the features:
Z_train = mapper.fit_transform(X_train)
Z_test = mapper.transform(X_test)

In [50]:
# Model
# With feature transformations complete, let's build a model. Import a regressor from sklearn:
from sklearn.linear_model import LinearRegression

# Fit it to the generated Z_train object:
model = LinearRegression()
model.fit(Z_train, y_train)

# And score to confirm that everything works:
model.score(Z_test, y_test)

0.9155514997544354

In [ ]:
# Pipe
# While mapper and model are working, it's better if both are fit to a single, unified object.
# Combine the two objects into a scikit-learn pipeline with the make_pipeline function:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(mapper, model)

# Fit the new pipe object:
pipe.fit(X_train, y_train)
# Score it as though it were a scikit-learn estimator:
print(pipe.score(X_test, y_test))

# And confirm that the unified pipe object can handle a new, unseen, diamond:
new = pd.DataFrame({
    'carat': [2],
    'cut': ['Ideal'],
    'color': ['E'],
    'clarity': ['VVS1'],
})

print(pipe.predict(new))

0.9170453756606453
[16270.]


In [ ]:
# Pickle
# With mapper and model contained in a single object, it's super easy to serialize (read: save) the pipeline.
# Import pickle to start serialization process:
import pickle

# Dump the python objection with pickle.dump:
pickle.dump(pipe, open('pipe.pkl', 'wb'))
# reload the pipeline with pickle.load:
new_pipe = pickle.load(open('pipe.pkl', 'rb'))
# And confirm that the pipeline still works:
new_pipe.predict(new)[0]